In [1]:
from pathlib import Path
from typing import List, Union
from haystack import Pipeline, component
from haystack.dataclasses import Document, ChatMessage
from haystack.utils import Secret
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack_integrations.components.retrievers.chroma import ChromaQueryTextRetriever
from rank_bm25 import BM25L
import pickle
import fitz        # PyMuPDF для быстрого текстового извлечения
import nltk
from nltk.tokenize import word_tokenize

# Компоненты

In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# 2.1.  Классификатор: определяет, нужен ли поиск (search) или можно сразу answer
# ──────────────────────────────────────────────────────────────────────────────
@component
class QueryClassifier:
    @component.output_types(need_search=bool)
    def run(self, query: str) -> dict:
        # Простая эвристика: если в тексте есть вопросительный знак → поиск
        need_search = "?" in query.strip()
        return {"need_search": need_search}

In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# 2.2.  Декомпозитор: разбивает сложный запрос на подзапросы (если нужно)
#          В этом примере – если в запросе " и " или ',' → разбиваем по этим разделителям,
#          иначе возвращаем оригинал.
# ──────────────────────────────────────────────────────────────────────────────
@component
class QueryDecomposer:
    @component.output_types(subqueries=List[str])
    def run(self, query: str) -> dict:
        # очень простая декомпозиция
        if " и " in query or "," in query:
            parts = [part.strip() for part in query.replace(",", " и ").split(" и ") if part.strip()]
            return {"subqueries": parts}
        else:
            return {"subqueries": [query]}

In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# 2.3.  Retriever‑компонент на основе BM25 через pickle
# ──────────────────────────────────────────────────────────────────────────────
@component
class PickledBM25Retriever:
    @component.output_types(documents=List[Document])
    def __init__(self, path_to_pickle: str = "../data/bm25.pkl", top_k: int = 5):
        # Загружаем корпус и BM25
        with open(path_to_pickle, "rb") as f:
            self.bm25, self.doc_ids = pickle.load(f)
        # Предполагаем, что список doc_ids соотнесён по индексу с первым пайплайном
        # и что ChromaDocumentStore хранит документы с теми же id.
        self.top_k = top_k

    def run(self, query: str) -> dict:
        # токенизируем
        tokens = word_tokenize(query.lower())
        scores = self.bm25.get_scores(tokens)
        # берём top_k
        top_n = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[: self.top_k]
        # создаём Document-объекты с id и пустым content (контент подтянет Chroma)
        docs = [Document(id=self.doc_ids[i], content="", meta={}) for i in top_n]
        return {"documents": docs}

In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# 2.4.  Гибридный Search: BM25 + ChromaQueryTextRetriever → объединяем
# ──────────────────────────────────────────────────────────────────────────────
@component
class HybridRetriever:
    @component.output_types(documents=List[Document])
    def __init__(self, bm25_retriever: PickledBM25Retriever, chroma_retriever: ChromaQueryTextRetriever):
        self.bm25 = bm25_retriever
        self.chroma = chroma_retriever

    def run(self, query: str) -> dict:
        docs_bm = self.bm25.run(query)["documents"]
        docs_ch = self.chroma.run(query=query)["documents"]
        # объединяем, сохраняем уникальность
        combined = {d.id: d for d in docs_bm + docs_ch}
        return {"documents": list(combined.values())}

In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# 2.5.  PromptBuilder: формирует единую строку prompt
# ──────────────────────────────────────────────────────────────────────────────
prompt_template = """
Ты — умный помощник. Отвечай по-русски, используй фрагменты из контекста.
Вопрос: {{ query }}

Контекст:
{% for doc in documents %}
--- {{doc.meta.name or doc.id}} ---
{{ doc.content }}
{% endfor %}

Ответ:
"""
prompt_builder = PromptBuilder(
    template=prompt_template,
    required_variables=["query", "documents"]
)

In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# 2.6.  Инициализация LLM-генератора (Saiga 12B через Ollama API)
# ──────────────────────────────────────────────────────────────────────────────
MODEL_NAME = "hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q8_0"
llm_gen = OpenAIChatGenerator(
    model=MODEL_NAME,
    api_key=Secret.from_token("ollama"),
    api_base_url="http://localhost:11434/v1",
    generation_kwargs={"temperature": 0.8}
)

In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# 2.7.  Собираем Pipeline 2
# ──────────────────────────────────────────────────────────────────────────────
pipeline = Pipeline()

# 1) входной узел — принимает {'query': str}
# 2) classifier → определяет, нужен поиск или нет
pipeline.add_component("classifier", QueryClassifier())
pipeline.add_component("router", component(_fast_path=None))  # placeholder

# Вместо ConditionalRouter — простой Python‑роутер
@component
class QueryRouter:
    @component.output_types(search=str, direct=str)
    def run(self, query: str, need_search: bool) -> dict:
        if need_search:
            return {"search": query, "direct": ""}
        else:
            return {"search": "",   "direct": query}

pipeline.add_component("query_router", QueryRouter())

# 3) декомпозиция сложных запросов
pipeline.add_component("decomposer", QueryDecomposer())

# 4) ретриверы
bm25_r = PickledBM25Retriever(path_to_pickle="../data/bm25.pkl", top_k=5)
chroma_r = ChromaQueryTextRetriever(
    collection_name="documents",
    persist_directory="data/chroma_index",
    embedding_model="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    top_k=5
)
pipeline.add_component("hybrid_retriever", HybridRetriever(bm25_r, chroma_r))

# 5) prompt builder & генератор
pipeline.add_component("prompt_builder", prompt_builder)
pipeline.add_component("generator", llm_gen)

In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# 2.8.  Подключения
# ──────────────────────────────────────────────────────────────────────────────
#  вход: {'query': ...}
pipeline.connect("query", "classifier.query")
pipeline.connect("query", "query_router.query")
pipeline.connect("classifier.need_search", "query_router.need_search")

# путь «прямой ответ» (direct) → пропускаем поиск → сразу Prompt+LLM
pipeline.connect("query_router.direct", "prompt_builder.query")
pipeline.connect("query_router.direct", "prompt_builder.documents")  # пустой список
pipeline.connect("prompt_builder.prompt", "generator.messages", sender_role="user")

# путь «с поиском» (search) → декомпозиция → hybrid_retriever → Prompt+LLM
# note: если пустой search, HybridRetriever просто не найдёт документов
pipeline.connect("query_router.search",    "decomposer.query")
pipeline.connect("decomposer.subqueries", "hybrid_retriever.query")
pipeline.connect("hybrid_retriever.documents", "prompt_builder.documents")
pipeline.connect("query_router.search",    "prompt_builder.query")

# генерация чат‑сообщения
# Преобразуем prompt (строку) в ChatMessage
@component
class ToChatMessage:
    @component.output_types(messages=List[ChatMessage])
    def run(self, prompt: str) -> dict:
        return {"messages": [ChatMessage(role="user", content=prompt)]}

pipeline.add_component("to_chat", ToChatMessage())
pipeline.connect("prompt_builder.prompt", "to_chat.prompt")
pipeline.connect("to_chat.messages",       "generator.messages")

In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# 3. Запуск и проверка
# ──────────────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    # Пример 1: простой direct‑запрос
    out = pipeline.run({"query": "Привет, как дела?"})
    print("Direct:", out["generator.replies"])

    # Пример 2: информационный запрос
    out = pipeline.run({"query": "Какие новые политики отпуска и сколько дней теперь положено?"})
    print("Answer:", out["generator.replies"])